In [4]:
import os
import numpy as np
import tensorflow as tf

In [5]:
def model(inputs, train = True, norm = True, **kwargs):
    '''
    '''
    def herp():
        return 1
    pass
#     return outputs, {}

AttributeError: 'function' object has no attribute 'herp'